In [1]:
def knapsack(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, capacity + 1):
            if weights[i - 1] <= j:
                dp[i][j] = max(dp[i - 1][j], values[i - 1] + dp[i - 1][j - weights[i - 1]])
            else:
                dp[i][j] = dp[i - 1][j]

    return dp


# 示例用法
values = [60, 100, 120]  # 物品价值
weights = [10, 20, 30]  # 物品重量
capacity = 50  # 背包容量

max_value = knapsack(values, weights, capacity)
print("背包中能装入的物品的最大总价值为:", max_value)

背包中能装入的物品的最大总价值为: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 220]]


In [4]:
import numpy as np
print(np.array(max_value))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  60  60  60  60  60  60  60  60
   60  60  60  60  60  60  60  60  60  60  60  60  60  60  60  60  60  60
   60  60  60  60  60  60  60  60  60  60  60  60  60  60  60]
 [  0   0   0   0   0   0   0   0   0   0  60  60  60  60  60  60  60  60
   60  60 100 100 100 100 100 100 100 100 100 100 160 160 160 160 160 160
  160 160 160 160 160 160 160 160 160 160 160 160 160 160 160]
 [  0   0   0   0   0   0   0   0   0   0  60  60  60  60  60  60  60  60
   60  60 100 100 100 100 100 100 100 100 100 100 160 160 160 160 160 160
  160 160 160 160 180 180 180 180 180 180 180 180 180 180 220]]
